In [2]:
from pathlib import Path
import os
from openai import OpenAI
import numpy as np
from dotenv import load_dotenv
load_dotenv()
api_key  = os.getenv('OPENAI_APIKEY')
# APIクライアントの初期化

client = OpenAI(
    # This is the default and can be omitted
    api_key=api_key,
)

In [4]:
command=f"""
curl https://api.openai.com/v1/embeddings \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer {api_key}" \
"""
command+=""" \
  -d '{
    "input": "Your text string goes here",
    "model": "text-embedding-3-small"
  }'
"""
output=!$command

In [20]:
print(output)

['{', '  "object": "list",', '  "data": [', '    {', '      "object": "embedding",', '      "index": 0,', '      "embedding": [', '        0.005132983,', '        0.017242905,', '        -0.018698474,', '        -0.018558515,', '        -0.047250036,', '        -0.03028705,', '        0.027669823,', '        0.003614432,', '        0.011217685,', '        0.0064345985,', '        -0.0016865017,', '        0.015815327,', '        -0.0012849952,', '        -0.007858678,', '        0.0599023,', '        0.05035712,', '        -0.02754386,', '        0.00991607,', '        -0.040392064,', '        0.049993224,', '        -0.00038597969,', '        0.030231066,', '        -0.013729943,', '        0.032974254,', '        0.017312884,', '        0.016809033,', '        -0.0017661031,', '        0.020391975,', '        0.04081194,', '        -0.03770486,', '        -0.026172265,', '        -0.050021216,', '        0.024184853,', '        -0.055199686,', '        -0.03224647,', '        0.04232

### embeddingを理解するために，いくつかの文書を入れる



In [22]:
data = [
    {
        "university": "東大",
        "features": "日本を代表する大学で、幅広い学問分野で世界的に高い評価を受けています。研究と教育の水準が非常に高いのが特徴です。"
    },
    {
        "university": "東工大",
        "features": "理工系の専門性に特化した大学で、技術革新や最先端研究に強みがあります。工学分野での世界的な評価も高いです。"
    },
    {
        "university": "東京理科大",
        "features": "実践的な理系教育を重視し、産業界で活躍する人材を輩出しています。理系特化のカリキュラムが魅力です。"
    },
    {
        "university": "早稲田大",
        "features": "多様な学問分野を持ち、国際色豊かな教育を提供しています。自由な校風とグローバルな視点が魅力です。"
    },
    {
        "university": "慶応大学",
        "features": "日本初の私立大学で、ビジネスや法学分野に強みがあります。ネットワークの広さと卒業生の活躍が特筆されます。"
    }
]


In [41]:
index = []
for datum in data:
    # ベクトル化
    vec = client.embeddings.create(
      model='text-embedding-3-small',
      input=datum["features"]
    )
#    print(vec)

    # ベクトルをリストに追加
    index.append({
        'title': datum['university'],
        'body': datum['features'],
        'embedding': vec.data[0].embedding
    })

In [43]:
input = "私は実践的な技術の勉強をしたいです。"

vec = client.embeddings.create(
    model='text-embedding-3-small',
    input=input
)
input_vec = vec.data[0].embedding

In [45]:
def cosine_similarity(x, y):
    dot_product = np.dot(x, y)
    norm_x = np.linalg.norm(x)
    norm_y = np.linalg.norm(y)
    # コサイン類似度を計算
    similarity = dot_product / (norm_x * norm_y)
    return similarity


In [53]:
# 総当りで類似度を計算
results = map(
        lambda i: {
            'title': i['title'],
            'body': i['body'],
            # ここで入力とのコサイン類似度を計算
            'similarity': cosine_similarity(i['embedding'], input_vec)
            },
        index
)

In [55]:
results = sorted(results, key=lambda i: i['similarity'], reverse=True)


In [56]:
print(f'title: {results[0]["title"]}')
print(f'body: {results[0]["body"]}')

title: 東京理科大
body: 実践的な理系教育を重視し、産業界で活躍する人材を輩出しています。理系特化のカリキュラムが魅力です。
